In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

In [16]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

In [17]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = set(text)
    vocab_to_int, int_to_vocab = {}, {}
    for i, w in enumerate(vocab):
        vocab_to_int[w] = i
        int_to_vocab[i] = w
    return (vocab_to_int, int_to_vocab)    


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


In [18]:
def token_lookup():
     
    token={'.':'||Period||',
           ',':'||Comma||',
           '"':'||Quotationmark||',
           ';':'||Semicolon||',
           '!':'||Exclamationmark||',
           '(':'||LeftParentheses||',
           ')':'||RightParentheses||',
           '-':'||Dash||',
           '\n':'||Return||',
           '?':'||Questionmark||'}
    return token

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


In [19]:

helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

In [1]:

import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [2]:
import torch
train_on_gpu = torch.cuda.is_available()

In [3]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

def batch_data(words, sequence_length, batch_size):
    feature_tensors=[]
    target_tensors=[]
    for i in range(0,len(words)-1):
        if i+sequence_length+1>len(words)-1:
            break
        feature_tensors.append(words[i:i+sequence_length])
        target_tensors.append(words[i+sequence_length])
    feature_tensors=np.array(feature_tensors)
    target_tensors=np.array(target_tensors)
    feature_tensors=torch.from_numpy(feature_tensors)
    target_tensors=torch.from_numpy(target_tensors)
    data=TensorDataset(feature_tensors,target_tensors)   
    data_loader=DataLoader(data,batch_size=batch_size)
        
    
   
    return data_loader

In [4]:
test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  0,   1,   2,   3,   4],
        [  1,   2,   3,   4,   5],
        [  2,   3,   4,   5,   6],
        [  3,   4,   5,   6,   7],
        [  4,   5,   6,   7,   8],
        [  5,   6,   7,   8,   9],
        [  6,   7,   8,   9,  10],
        [  7,   8,   9,  10,  11],
        [  8,   9,  10,  11,  12],
        [  9,  10,  11,  12,  13]])

torch.Size([10])
tensor([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14])


In [5]:
import torch.nn as nn

class RNN(nn.Module):
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        
        super(RNN, self).__init__()
        
        self.vocab_size=vocab_size
        self.output_size=output_size
        self.embedding_dim=embedding_dim
        self.hidden_dim=hidden_dim
        self.n_layers=n_layers
        self.dropout=dropout
        self.embedd=nn.Embedding(self.vocab_size,self.embedding_dim)
        self.LSTM=nn.LSTM(self.embedding_dim,self.hidden_dim,self.n_layers,dropout=self.dropout,batch_first=True)
        self.fc1=nn.Linear(self.hidden_dim,output_size)
        
    
    def forward(self, nn_input, hidden):
        batch_size=nn_input.size(0)
        emdedds=self.embedd(nn_input)
        lstm_out,hidden=self.LSTM(emdedds,hidden)
        lstm_out=lstm_out.contiguous().view(-1,self.hidden_dim)
        output=self.fc1(lstm_out)
        outputF= output.view(batch_size, -1, self.output_size)
        outputF=outputF[:,-1]
        return outputF, hidden
    
    
    def init_hidden(self, batch_size):
       
        weight=next(self.parameters()).data
        
        if train_on_gpu:
            hidden=(weight.new(self.n_layers,batch_size,self.hidden_dim).zero_().cuda(),
                   weight.new(self.n_layers,batch_size,self.hidden_dim).zero_().cuda())
        else:
            hidden=(weight.new(self.n_layers,batch_size,self.hidden_dim).zero_(),
                   weight.new(self.n_layers,batch_size,self.hidden_dim).zero_())
        # initialize hidden state with zero weights, and move to GPU if available
        
        return hidden


tests.test_rnn(RNN, train_on_gpu)

Tests Passed


In [6]:
def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
   
    hidden = tuple([each.data for each in hidden])
    if train_on_gpu:
        inp=inp.cuda()
        rnn=rnn.cuda()
        target=target.cuda()
    output,hidden=rnn(inp,hidden)
    loss=criterion(output,target)
    optimizer.zero_grad()
    loss.backward()
    nn.utils.clip_grad_norm_(rnn.parameters(), 5)
    optimizer.step()
    
    return loss.item(), hidden

tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


In [7]:

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    for epoch_i in keep_awake(range(1, n_epochs + 1)):
        
       
        hidden = rnn.init_hidden(batch_size)
        
        for batch_i, (inputs, labels) in enumerate(train_loader, 1):
            
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break
            
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
            batch_losses.append(loss)

           
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                    epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []

    return rnn

In [8]:

sequence_length = 15  
batch_size = 128


train_loader = batch_data(int_text, sequence_length, batch_size)

In [9]:

num_epochs = 5

learning_rate = 0.001


vocab_size = len(vocab_to_int)

output_size = len(vocab_to_int)

embedding_dim = 400

hidden_dim = 500

n_layers = 3

show_every_n_batches = 1000

In [11]:
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

helper.save_model('./save/trained_rnn', trained_rnn)
print('Model Trained and Saved')

Training for 5 epoch(s)...
Epoch:    1/5     Loss: 5.831405641078949

Epoch:    1/5     Loss: 5.761553304195404

Epoch:    1/5     Loss: 5.904258048534393

Epoch:    1/5     Loss: 5.857336911201477

Epoch:    1/5     Loss: 4.801289212465286

Epoch:    1/5     Loss: 4.695549366474151

Epoch:    2/5     Loss: 4.443392631061916

Epoch:    2/5     Loss: 4.203418606042862

Epoch:    2/5     Loss: 4.281730809211731

Epoch:    2/5     Loss: 4.144045569419861

Epoch:    2/5     Loss: 3.9333086996078492

Epoch:    2/5     Loss: 4.040463302612305

Epoch:    3/5     Loss: 3.939927311098594

Epoch:    3/5     Loss: 3.8128743925094604

Epoch:    3/5     Loss: 3.9036639001369475

Epoch:    3/5     Loss: 3.82889416384697

Epoch:    3/5     Loss: 3.6628991487026217

Epoch:    3/5     Loss: 3.772681785583496

Epoch:    4/5     Loss: 3.6849228270227017

Epoch:    4/5     Loss: 3.585073459148407

Epoch:    4/5     Loss: 3.64347318649292

Epoch:    4/5     Loss: 3.590347488641739

Epoch:    4/5     Loss: 

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


In [12]:

import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn')

In [13]:
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    rnn.eval()
    
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        hidden = rnn.init_hidden(current_seq.size(0))
        
        output, _ = rnn(current_seq, hidden)
        
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() 
         
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    return gen_sentences

In [14]:

gen_length = 400 
prime_word = 'jerry' 
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:41: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().


jerry: fails frequent frequent fails fails fails fails fails fails pray.

george: i think we could be prosecuted for the defendants.

jerry: i know, i have to be able to get my own money.

george: what are you doing?

hoyt: what are you doing for?

hoyt: yes. the whole, the judge is the only time that was the first, but we had to do that. you know how to be the defendants of the lipo?

vandelay: i have to go to a lot of trivial of the courtroom, you know, they were going to be able to create perry of these manner, i got it in the house, and we were in the plane.

george: no.

[new witness: the robbery]

jerry: what?

george: yes, i think you were.

elaine: yes, i think we could have to be able to create it.

elaine: you don't have to get to be the contest.

jerry: oh yeah..

[new nazi: yev donald.

hoyt: i know.

chiles: what?

hoyt: i think it was in mortal uniforms.

elaine: oh, no.

hoyt: so, i was thinking about that term.

vandelay: you know, i don't have to tell him about a lot o

In [15]:
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

In [10]:
import signal
 
from contextlib import contextmanager
 
import requests
 
 
DELAY = INTERVAL = 4 * 60  # interval time in seconds
MIN_DELAY = MIN_INTERVAL = 2 * 60
KEEPALIVE_URL = "https://nebula.udacity.com/api/v1/remote/keep-alive"
TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
TOKEN_HEADERS = {"Metadata-Flavor":"Google"}
 
 
def _request_handler(headers):
    def _handler(signum, frame):
        requests.request("POST", KEEPALIVE_URL, headers=headers)
    return _handler
 
 
@contextmanager
def active_session(delay=DELAY, interval=INTERVAL):
    """
    Example:
 
    from workspace_utils import active_session
 
    with active_session():
        # do long-running work here
    """
    token = requests.request("GET", TOKEN_URL, headers=TOKEN_HEADERS).text
    headers = {'Authorization': "STAR " + token}
    delay = max(delay, MIN_DELAY)
    interval = max(interval, MIN_INTERVAL)
    original_handler = signal.getsignal(signal.SIGALRM)
    try:
        signal.signal(signal.SIGALRM, _request_handler(headers))
        signal.setitimer(signal.ITIMER_REAL, delay, interval)
        yield
    finally:
        signal.signal(signal.SIGALRM, original_handler)
        signal.setitimer(signal.ITIMER_REAL, 0)
 
 
def keep_awake(iterable, delay=DELAY, interval=INTERVAL):
    """
    Example:
 
    from workspace_utils import keep_awake
 
    for i in keep_awake(range(5)):
        # do iteration with lots of work here
    """
    with active_session(delay, interval): yield from iterable